# Chapter3. 스파크 기능 둘러보기

스파크는 기본 요소인 저수준 API와 구조적 API 그리고 추가 기능을 제공하는 일련의 표준 라이브러리로 구성되어 있다.

스파크의 라이브러리는 그래프 분석, 머신러닝 그리고 스트리밍 등 다양한 작업을 지원하며, 컴퓨팅 및 스토리지 시스템과의 통합을 돕는 역할을 수행한다. 

<학습할 내용>
* spark-submit 명령문으로 운영용 애플리케이션 실행
* Dataset : 타입 안전성을 제공하는 구조적 API
* 구조적 스트리밍
* 머신러닝과 고급 분석
* RDD : 스파크의 저수준 API
* SparkR
* 서드파티 패키지 에코시스템

## 3.1 운영용 애플리케이션 실행하기

spark-submit 명령을 사용해 대화형 셀에서 개발한 프로그램을 운영용 애플리케이션으로 쉽게 전환 가능. 

* spark-submit 명령 : 애플리케이션 코드를 클러스터에 전송해 실행시키는 역할. (애플리케이션 실행에 필요한 자원과 실행 방식 그리고 다양한 옵션을 지정 가능)

클러스터에 제출된 애플리케이션은 작업이 종료되거나 에러가 발생할 때까지 실행

## 3.2 Dataset : 타입 안전성을 제공하는 구조적 API

Dataset은 자바와 스칼라의 정적 데이터 타입에 맞는 코드 즉 정적 타입 코드를 지원하기 위함. 따라서 동적 타입 언어인 파이썬과 R에서는 사용 불가능.

DataFrame은 다양한 데이터 타입의 테이블형 데이터를 보관할 수 있는 Row 타입의 객체로 구성된 분산 컬렉션. 

< Dataset API >
* Dataset API는 DataFrame의 레코드를 사용자가 자바나 스칼라로 정의한 클래스에 할당하고 자바의 ArrayList 또는 스칼라의 Seq 객체 등의 고정 타입형 컬렉션으로 다룰 수 있는 기능 제공
* Dataset API는 타입 안정성을 지원해 초기화에 사용한 클래스 대신 다른 클래스를 사용해 접근할 수 없으므로 다수의 소프트웨어 엔지니어가 잘 정의된 인터페이스로 상호작용하는 대규모 애플리케이션을 개발하는데 유용
* Dataset 클래스는 내부 객체의 데이터 타입을 매개변수로 사용 
* 필요한 경우에 선택적으로 사용 가능
* 데이터 타입을 정의하고 map과 filter 함수 사용 가능
* collect 메서드나 take 메서드를 호출하면 DataFrame을 구성하는 Row 타입의 객체가 아닌 Dataset에 매개변수로 지정한 타입의 객체를 반환하므로 코드 변경 없이 타입 안정성을 보장할 수 있고 로컬이나 분산 클러스터 환경에서 데이터를 안전하게 다룰 수 있다.

## 3.3 구조적 스트리밍

구조적 스트리밍 : 스파크 2.2 버전에서 안정화된 스트림 처리용 고수준 API

구조적 스트리밍을 사용하면
* 구조적 API로 개발된 배치 모드의 연산을 스트리밍 방식으로 실행 가능
* 지연 시간을 줄이고 증분 처리 가능
* 배치 처리용 코드를 일부 수정해 스트리밍 처리를 수행하고 값을 빠르게 얻을 수 있음
* 프로토타입을 배치 잡으로 개발한 다음 스트리밍 잡으로 변환할 수 있어 개념 잡기가 수월

예제) 소매(retail) 데이터셋 중 하루치 데이터를 나타내는 by-day 디렉터리 파일 사용

In [0]:
staticDataFrame = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferShema", "true")\
  .load('/FileStore/tables/data/retail-data/by-day/*.csv')

staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema

시계열 데이터를 다루기 때문에 데이터를 그룹화하고 집계하는 방법

예시) 특정 고객(CustomerId로 구분)이 대량으로 구매하는 영업 시간을 살펴보기 위해 총 구매비용 컬럼을 추가하고 고객이 가장 많이 소비한 날 찾아보기

In [0]:
from pyspark.sql.functions import window, col

staticDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")\
  .show(5)

+----------+--------------------+------------------+
CustomerId| window| sum(total_cost)|
+----------+--------------------+------------------+
 15274.0|{2011-12-05 00:00...| 665.1600000000002|
 14719.0|{2011-12-08 00:00...| 812.8399999999998|
 16794.0|{2011-12-08 00:00...|201.32000000000002|
 12464.0|{2011-11-29 00:00...| 563.8|
 15269.0|{2011-11-16 00:00...| 817.6|
+----------+--------------------+------------------+
only showing top 5 rows

null 값을 가진 결과의 로우는 일부 트랜잭션에서 customerId 값이 없음을 의미.

스트리밍 코드는 read 메서드 대신 readStream 메서드를 사용. 그리고 maxFilesPerTrigger 옵션을 추가로 지정해 한 번에 읽을 파일 수 설정 가능.

In [0]:
streamingDataFrame = spark.readStream\
  .schema(staticSchema)\
  .option("maxFilesPerTrigger", 1)\
  .format("csv")\
  .option("header", "true")\
  .load("/FileStore/table/data/retail-data/by-day/*.csv")

In [0]:
# DataFrame이 스트리밍 유형인지 확인 - true/false 반환
streamingDataFrame.isStreaming  

Out[14]: True

기존 DataFrame 처리와 동일한 비지니스 로직인지 적용

예시) 총 판매 금액 계산

In [0]:
purchaseByCustomerPerHour = streamingDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")

지연 연산이므로 데이터 플로를 실행하기 위해 스트리밍 액션 호출 필수.

스트리밍 액션은 어딘가에 데이터를 채워 넣어야 하므로 count 메서드와 같은 일반적인 정적 액션과는 조금 다른 특성을 가진다. 여기서 사용하는 스트리밍 액션은 트리거가 실행된 다음 데이터를 갱신하게 될 인메모리 테이블에 데이터를 저장. 

(스파크는 이전 집계값보다 더 큰 값이 발생한 경우에만 인메모리 테이블을 갱신하므로 언제나 가장 큰 값을 얻을 수 있다.)

In [0]:
purchaseByCustomerPerHour.writeStream\
  .format("memory")\
  .queryName("customer_purchases")\
  .outputMode("complete")\
  .start()

Out[16]: <pyspark.sql.streaming.StreamingQuery at 0x7fa36e530d60>

스트림이 시작되면 쿼리 실행 결과가 어떠한 형태로 인메모리 테이블에 기록되는지 확인 가능

In [0]:
spark.sql("""
  SELECT *
  FROM customer_purchases
  ORDER BY 'sum(total_cost)' DESC
  """)\
  .show(5)

+----------+------+---------------+
CustomerId|window|sum(total_cost)|
+----------+------+---------------+
+----------+------+---------------+

## 3.4 머신러닝과 고급 분석

* 스파크에서는 내장된 머신러닝 라이브러리인 MLlib을 사용해 대규모 머신러닝 수행 가능. 
* MLlib을 사용하면 대용량 데이터를 대상으로 전처리, 멍잉(데이터 랭클링, 원본 데이터를 다른 형태로 변환 혹은 매핑하는 과정 의미), 모델 학습 및 예측이 가능.
* 구조적 스트리밍에서 예측하고자 할 때도 MLlib에서 학습시킨 다양한 예측 모델 사용 가능.
* 스파크는 분류(classification), 회귀(regression), 군집화(clustering), 딥러닝(deep learning)까지 머신러닝과 관련된 정교한 API 제공
* 스파크는 데이터 전처리에 사용하는 다양한 메서드 제공

k-평균이라는 표준 알고리즘을 이용한 기본적인 군집화 수행

예제) 원본 데이터를 올바른 포맷으로 만드는 트랜스포메이션을 정의하고 실제로 모델을 학습해 예측 수행

In [0]:
staticDataFrame.printSchema()

root
-- InvoiceNo: string (nullable = true)
-- StockCode: string (nullable = true)
-- Description: string (nullable = true)
-- Quantity: string (nullable = true)
-- InvoiceDate: string (nullable = true)
-- UnitPrice: string (nullable = true)
-- CustomerID: string (nullable = true)
-- Country: string (nullable = true)

MLlib의 머신러닝 알고리즘을 사용하기 위해서는 수치형 데이터가 필요하므로 예제의 데이터에서 타임스탬프, 정수, 문자열 타입의 데이터를 수치형 데이터로 변환 과정 필요.

In [0]:
# DataFrame 트랜스포메이션을 사용해 날짜 데이터를 다루는 예제

from pyspark.sql.functions import date_format, col

preppedDataFrame = staticDataFrame\
  .na.fill(0)\
  .withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))\
  .coalesce(5)

데이터를 학습 데이터셋과 테스트 데이터셋으로 분리

* 예제에서는 특정 구매가 이루어진 날짜를 기준으로 직접 분리
* MLlib의 트랜스포메이션 API(TrainValidationSplit 혹은 CrossValidator)를 사용해 학습 데이터셋과 테스트 데이터셋을 생성 가능

In [0]:
trainDataFrame = preppedDataFrame\
  .where("InvoiceDate < '2011-07-01'")

testDataFrame = preppedDataFrame\
  .where("InvoiceDate >= '2011-07-01'")

액션을 호출해 데이터 분리

예제의 데이터는 시계열 데이터셋으로 임의 날짜를 기준으로 데이터를 분리한다.

In [0]:
trainDataFrame.count()
testDataFrame.count()

Out[24]: 592012

스파크 MLlib은 일반적인 트랜스포메이션을 자동화하는 다양한 트랜스포메이션 제공하는데 그 중 하나가 StringIndexer

In [0]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer()\
  .setInputCol("day_of_week")\
  .setOutputCol("day_of_index")

앞에서는 요일을 수치형(토요일은 6, 월요일은 1, ...)으로 반환해 암묵적으로 토요일이 월요일보다 크다는 것을 의미하게 된다.

따라서 OneHotEncoder를 사용해 각 값을 자체 컬럼으로 인코딩해야 한다. 이렇게 하면 특정 요일이 해당 요일인지 아닌지 불리언 타입으로 나타낼 수 있다.

In [0]:
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder()\
  .setInputCol("day_of_week_index")\
  .setOutputCol("day_of_woeek_encoded")

위의 결과는 벡터 타입을 구성할 컬럼 중 하나로 사용.

In [0]:
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler()\
  .setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"])\
  .setOutputCol("features")

위의 예제는 가격, 수량, 특정 날짜의 요일을 가지고 있다. 다음은 나중에 입력 값으로 들어올 데이터가 같은 프로세스를 거쳐 변환되도록 파이프라인을 설정

In [0]:
from pyspark.ml import Pipeline

transformationPipeline = Pipeline()\
  .setStages([indexer, encoder, vectorAssembler])

모델 학습 준비 과정

* 변환자를 데이터셋에 적합
* StringIndexer는 인덱싱할 고윳값의 수를 알아야 함. 알 수 없다면 컬럼에 있는 모든 고윳값을 조사하고 인덱싱

In [0]:
fittedPipeline = transformationPipeline.fit(trainDataFrame)

--------------------------------------------------------------------------- 
 IllegalArgumentException Traceback (most recent call last)
 <command-2861523922385080> in <module> 
 ----> 1 fittedPipeline = transformationPipeline . fit ( trainDataFrame ) 

 /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py in patched_method (self, *args, **kwargs) 
 28 call_succeeded = False 
 29 try : 
 ---> 30 result = original_method ( self , * args , ** kwargs ) 
 31 call_succeeded = True 
 32 return result

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 159 return self . copy ( params ) . _fit ( dataset ) 
 160 else : 
 --> 161 return self . _fit ( dataset ) 
 162 else : 
 163 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/pipeline.py in _fit (self, dataset) 
 112 dataset = stage . transform ( dataset ) 
 113 else : # must be an Estimator 
 --> 114 model = stage . fit ( dataset ) 
 115 transformers . append ( model ) 
 116 if i < indexOfLastEstimator : 

 /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py in patched_method (self, *args, **kwargs) 
 28 call_succeeded = False 
 29 try : 
 ---> 30 result = original_method ( self , * args , ** kwargs ) 
 31 call_succeeded = True 
 32 return result

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 159 return self . copy ( params ) . _fit ( dataset ) 
 160 else : 
 --> 161 return self . _fit ( dataset ) 
 162 else : 
 163 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 333 
 334 def _fit ( self , dataset ) : 
 --> 335 java_model = self . _fit_java ( dataset ) 
 336 model = self . _create_model ( java_model ) 
 337 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 330 """
 331 self . _transfer_params_to_java ( ) 
 --> 332 return self . _java_obj . fit ( dataset . _jdf ) 
 333 
 334 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 IllegalArgumentException : day_of_week_index does not exist. Available: InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country, day_of_week, day_of_index

학습 데이터셋에 변환자를 적합시키면 학습을 위한 파이프라인 준비. 이를 이용해 일관되고 반복적인 방식으로 모든 데이터 변환 가능

In [0]:
transformedTraining = fittedPipeline.transform(trainDataFrame)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-446413481131067> in <module> 
 ----> 1 transformedTraining = fittedPipeline . transform ( trainDataFrame ) 

 NameError : name 'fittedPipeline' is not defined

모델을 학습하기 위해서 관련 클래스 import 및 인스턴스 생성

In [0]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans()\
  .setK(20)\
  .setSeed(1L)

In [0]:
kmModel = kmeans.fit(transformedTraining)

# 성과 지표에 따라 학습 데이터셋에 대한 비용 계산
transformedTest = fittedPipeline.transform(testDataFrame)
kmModel.computeCost(transformedTest)

## 3.5 저수준 API

스파크는 RDD를 통해 자바와 파이썬 객체를 다루는 데 필요한 기본 기능인 저수준 API를 제공하고, 거의 모든 기능은 RDD를 기반으로 만들어졌다. 

DataFrame 연산도 RDD를 기반으로 만들어졌으며 편리하고 효율적인 분산 처리를 위해 저수준 명령으로 컴파일. 

예제) 간단한 숫자를 이용해 병렬화해 RDD 생성 후 다른 DataFrame과 함께 사용할 수 있도록 DataFrame으로 변환

In [0]:
from pyspark.sql import Row

spark.sparkContext.parallelize([Row(1), Row(2), Row(3)]).toDF()

Out[39]: DataFrame[_1: bigint]

## 3.6 SparkR

SparkR은 스파크를 R 언어로 사용하기 위한 기능으로 스파크가 지원하는 모든 언어에 적용된 원칙을 동일하게 따른다. 

SparkR을 사용하기 위해서는 SparkR을 설치하고 코드를 실행하면 된다.